In [1]:
def __clear_env():
    for key in globals().keys():
        if not key.startswith("__"):# 排除系统内建函数
            globals().pop(key)
__clear_env
import example_pendulum
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
#import torchvision
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import csv
import optuna
from torch.autograd import Variable

3.69503875515263e-54


In [2]:
device = 'cuda:0'
data = example_pendulum.get_pendulum_data(10)
image = data['x']
image_t = data['dx']
image_tt = data['ddx']
angle = data['z']
angle_t = data['dz']
angle_tt = data['ddz']
print(image.shape)

(5000, 2601)


In [3]:
environment = "laptop"
loss_log = []
params = {}
#params['learning_rate'] = trial.suggest_float('lr',0,1)
params['epochs'] = 170
params['batch_size'] = 50
if environment == 'laptop':
    params['root_dir'] =R'C:\Users\87106\OneDrive\sindy\progress'
elif environment == 'desktop':
    params['root_dir'] = R'E:\OneDrive\sindy\progress'
elif environment == 'server':
    params['root_dir'] = R'/home/stilrmy/Angle_extractor'


# save parameters
params['if_save'] = True
params['save_date'] = '4-7'
params['save_ver'] = '1'
#load parameters
params['if_load'] = False
params['load_date'] = '4-7'
params['load_ver'] = '1'
PATH = os.path.join(params['root_dir'], params['save_date'],params['save_ver'])
loading_path = os.path.join(params['root_dir'], params['load_date'],params['load_ver'],'model.pth')
print(loading_path)

C:\Users\87106\OneDrive\sindy\progress/4-7/1/model.pth


In [4]:
class angle_predict(nn.Module):
    def __init__(self):
        super(angle_predict, self).__init__()
        self.fc1 = nn.Linear(2601, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 64)
        self.fc5 = nn.Linear(64, 3)
    def forward(self, x):
        m = nn.ReLU()
        x = self.fc1(x)
        x = m(x)
        x = self.fc2(x)
        x = m(x)
        x = self.fc3(x)
        x = m(x)
        x = self.fc4(x)
        x = m(x)
        x = self.fc5(x) 
        return x
model = angle_predict()
if params['if_load'] == True:
    model.load_state_dict(torch.load(loading_path))
model = model.to(device)

In [5]:
def objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-8, 1e-1, log=True)
    opt = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
    #opt = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
    for epoch in range(params['epochs']):
        loss_sum = 0
        loss_angle_sum = 0
        loss_angle_t_sum = 0
        loss_angle_tt_sum = 0
        count = 0 
        model.train()
        for i in range(len(data['z'])//params['batch_size']):
            image_temp = image[i*params['batch_size']:(i+1)*params['batch_size'],:]
            angle_temp = angle[i*params['batch_size']:(i+1)*params['batch_size']]
            angle_t_temp = angle_t[i*params['batch_size']:(i+1)*params['batch_size']]
            angle_tt_temp = angle_tt[i*params['batch_size']:(i+1)*params['batch_size']]
            for j in range(image_temp.shape[0]):
                input = Variable(torch.tensor(image_temp[j,:],dtype=torch.float32).to(device))
                pre = model.forward(input)
                angle_true = torch.tensor(angle_temp[j],dtype=torch.float32).to(device)
                angle_t_true = torch.tensor(angle_t_temp[j],dtype=torch.float32).to(device)
                angle_tt_true = torch.tensor(angle_tt_temp[j],dtype=torch.float32).to(device)
                loss_angle = torch.abs(angle_true - pre[0])
                loss_angle_t = torch.abs(angle_t_true - pre[1])
                loss_angle_tt = torch.abs(angle_tt_true - pre[2])
                loss = loss_angle + loss_angle_t + loss_angle_tt
                loss_sum += loss
                loss_angle_sum += loss_angle
                loss_angle_t_sum += loss_angle_t
                loss_angle_tt_sum += loss_angle_tt
                count += 1
                opt.zero_grad()
                loss.backward()
                opt.step()
            model.eval()
        trial.report(loss, epoch)
        loss_log.append(loss_sum.item()/count)
        print('epoch: ', epoch+1, 'loss: ', loss_sum.item()/count,'\t loss_angle:', loss_angle_sum.item()/count, '\t loss_angle_t:',loss_angle_t_sum.item()/count,'\t loss_angle_tt:',loss_angle_tt_sum.item()/count)

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100, timeout=600)
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-04-07 16:29:36,640] A new study created in memory with name: no-name-b588eb51-c10d-44a8-930a-033dd57d67f2


epoch:  1 loss:  10.28190234375 	 loss_angle: 1.04825546875 	 loss_angle_t: 2.480905078125 	 loss_angle_tt: 6.75273671875
epoch:  2 loss:  10.281825 	 loss_angle: 1.0482197265625 	 loss_angle_t: 2.480906640625 	 loss_angle_tt: 6.75270390625
epoch:  3 loss:  10.28175703125 	 loss_angle: 1.0481869140625 	 loss_angle_t: 2.48088984375 	 loss_angle_tt: 6.75267109375
epoch:  4 loss:  10.2816859375 	 loss_angle: 1.04815439453125 	 loss_angle_t: 2.480886328125 	 loss_angle_tt: 6.75264375
epoch:  5 loss:  10.28163125 	 loss_angle: 1.048122265625 	 loss_angle_t: 2.480874609375 	 loss_angle_tt: 6.7526125
epoch:  6 loss:  10.2815484375 	 loss_angle: 1.0480890625 	 loss_angle_t: 2.4808796875 	 loss_angle_tt: 6.75258125
epoch:  7 loss:  10.281484375 	 loss_angle: 1.0480587890625 	 loss_angle_t: 2.480866796875 	 loss_angle_tt: 6.75255078125
epoch:  8 loss:  10.281425 	 loss_angle: 1.04802646484375 	 loss_angle_t: 2.4808595703125 	 loss_angle_tt: 6.752509375
epoch:  9 loss:  10.28135625 	 loss_angle: 

In [ ]:
def saving(model,PATH):
    if os.path.exists(PATH) == False:
        os.makedirs(PATH)
    model_PATH = os.path.join(PATH, 'model.pth')
    torch.save(model.state_dict(), model_PATH)
    params_PATH = os.path.join(PATH, 'params.txt')
    with open(params_PATH, 'w') as f:
        f.write(str(params))
        f.close()
    loss_PATH = os.path.join(PATH, 'loss_log.csv')
    with open(loss_PATH, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(loss_log)
    #fig_PATH = os.path.join(PATH, 'loss.png') 
    #plt.savefig(fig_PATH)
    print("data saved")
    return

In [ ]:
if params['if_save'] == True:
    saving(model,PATH)

In [ ]:
#plotting result
plt.plot(loss_log)
fig_PATH = os.path.join(PATH, 'loss.png') 
plt.savefig(fig_PATH, bbox_inches='tight', pad_inches = +0.1)
plt.show()